### Merger Rate of Black Holes in a Primordial Black Hole Cluster

###### Code Writed by Tadeo D.
###### Dedicated to the Moon of my life.
**References:** [arXiv:2105.14523](https://arxiv.org/pdf/2105.14523.pdf).

In [2]:
import numpy as np
import matplotlib.pyplot as pl
import matplotlib as mpl
import math
import seaborn as sns
from scipy import stats
import csv
from scipy import constants
from scipy.integrate import odeint
from tqdm import tqdm
from scipy import integrate
from scipy.integrate import quad, dblquad
from astropy.cosmology import Planck18 as cosmo# WMAP9 as cosmo #
from astropy import constants as const
import astropy.units as u
#from astropy.cosmology import WMAP9, z_at_value
import pandas as pd
#from mpl_toolkits.mplot3d import Axes3D
#from matplotlib import cm
#pl.rcParams['figure.figsize'] = (16, 9)
#pl.style.use('ggplot')
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
#
#from mlxtend.plotting import plot_linear_regression
from sympy import *
from sympy import simplify

In [3]:
H0 = cosmo.H(0)  #cosmo.H(0)  <Quantity 67.66 km / (Mpc s)>
G = const.G
c = const.c
z_r = 5000 #3450
z_eq = 3411
z_0 = 0
z_init = 6.900e9
rho_c0 = (3*(H0**2))/(8*math.pi*G)
rho_eq = cosmo.critical_density(3411)
rho_crit = cosmo.critical_density(0)
Omega_DM = cosmo.Odm(0)
Omega_M = cosmo.Om(0)
Omega_R = 8.4e-5
Omega_lmb = 1-Omega_M - Omega_R

$$\rho_{i}(r) = \rho_{0,i}\left(\frac{r}{r_{0}}\right)^{-1} \left(1 + \frac{r^{2}}{r^{2}_{0}}\right)^{-2}$$

In [4]:
N_i = 10**4 #M_i/m_i is the number of stars of mass m_i
M = 10**5 * u.M_sun#M is the total mass of the cluster
R = 1.*u.parsec #Scale factor that determines the core radius
v_rel = (G*M/R)**(1./2.)#The characteristic velocity of the PBHs
m_i = 10*u.M_sun
m_j = m_i
Energy = G*(m_i*N_i)**(2.)/((2*R)*(1*u.M_sun)) #According to the virial theorem, the total energy

print("N_i =\t", N_i)
print("M =\t", M)
print("R =\t", R)
print("v_rel =\t", v_rel.to(u.parsec* u.s**(-1)))
print("m_i =\t", m_i)
print("m_j =\t", m_j)
print("E =\t", Energy.to(u.parsec**2.*u.s**(-2.)))

N_i =	 10000
M =	 100000.0 solMass
R =	 1.0 pc
v_rel =	 6.7209397035645e-13 pc / s
m_i =	 10.0 solMass
m_j =	 10.0 solMass
E =	 2.2585515249474822e-20 pc2 / s2


In [14]:
sigma = 2*math.pi*(85*math.pi/(6*math.sqrt(2)))**(2./7.) * (G**(2.) * (m_i+m_j)**(10./7.) * m_i**(2./7.) * m_j**(2./7.))/(c**(10./7.)*v_rel**(18./7.))
#sigma = sigma.to(u.m**(32./7)/u.s**(18./7.))#.to(u.parsec**2)
#sigma.to(u.AU**2)
sigma.to(u.parsec**2.)

<Quantity 5.16749856e-13 pc2>

In [6]:
def f_i(E): ## eq. 19 shapiro
    return (24. * (2.)**(1./2.) * N_i * R**2. * E**(7./2.))/(7*(math.pi**(3.))*(G**5.)*(M**5.))

def rho_i(r):## eq. 20
    return ((3*M)/(4*(math.pi)*(R**(3.))))*((1 + r**(2.)/R**(2.))**(-5./2.))

def phi(r):## eq. 21
    return (G*M/R)*((1 + r**(2.)/R**(2.))**(-1./2.))

def integrant(E):
    return E**(7./2.)*((2*(a - E))**(3./2.))

In [7]:
f_i(Energy)
#PH_i(1.*u.parsec)

<Quantity 2.5349244e+27 kg(3/2) s3 / (m(9/2) pc(3/2) solMass(3/2))>

In [8]:
n = (3.*N_i)/(4*math.pi*(R**(3.)))
print(n)

2387.32414637843 1 / pc3


In [10]:
r = np.linspace(.1, 1., 10)
print("r(pc)\t\t phi(r) \t\t\t\t\t\t v_i(r)")
phi_array=[]
for i in r:
    i=i*u.parsec
    rho_value = rho_i(i)
    phi_value = phi(i).to(u.parsec**(2.)/u.s**2)
    #print(rho_i(i), "\t", phi_value)
    phi_array.append(phi_value.value)
    a = phi_value.value
    I, err = quad(integrant, 0, a)
    constant = ((4.*math.pi*m_i)/rho_value)
    constant2 = (24. * (2.**(1./2.)) * N_i * R**(2.))/(7.*(math.pi**(3.))*(G**5.)*(M**5.))
    const = (constant*constant2).to(u.s**(10)/u.m**(10))
    vel = (I*const*(u.parsec**(12.)/(u.s**(12.)))).to(u.km**(2.)/(u.s**(2.)))
    result = ((7*math.pi*(phi_value**6.))/(2.**(17./2.)))
    vel2 = (result*const).to(u.km**(2.)/(u.s**(2.)))
    print("%.2f" %i.value,"\t",phi_value,"\t",(vel**(1./2.)),"\t", (vel2**(1./2.)))
print("\n")
#print(phi_array)

r(pc)		 phi(r) 						 v_i(r)
0.10 	 4.494685526656458e-25 pc2 / s2 	 14.628007517514401 km / s 	 14.628008469546558 km / s
0.20 	 4.429383960831521e-25 pc2 / s2 	 14.521356365337876 km / s 	 14.521357310428876 km / s
0.30 	 4.326600034242027e-25 pc2 / s2 	 14.351883283354884 km / s 	 14.351884217416094 km / s
0.40 	 4.194024892154189e-25 pc2 / s2 	 14.130288239919008 km / s 	 14.130289159558185 km / s
0.50 	 4.0402197923747064e-25 pc2 / s2 	 13.868772209234823 km / s 	 13.868773111853798 km / s
0.60 	 3.8733839099994897e-25 pc2 / s2 	 13.579406564867433 km / s 	 13.57940744865367 km / s
0.70 	 3.7005550067478666e-25 pc2 / s2 	 13.272995623621165 km / s 	 13.272996487465312 km / s
0.80 	 3.527264880702962e-25 pc2 / s2 	 12.958494725490082 km / s 	 12.958495568865619 km / s
0.90 	 3.357536254982149e-25 pc2 / s2 	 12.642875887612925 km / s 	 12.642876710447092 km / s
1.00 	 3.1940741978991656e-25 pc2 / s2 	 12.331276031101156 km / s 	 12.331276833655526 km / s




In [80]:
def n_i(r):
    return (rho_i(r)/m_i)

def n_j(r2):
    return (rho_i(r2)/m_j)

def vel_j(r):
    constant = ((4.*math.pi*m_j)/rho_i(r))
    constant2 = (24. * (2.**(1./2.)) * N_i * R**(2.))/(7.*(math.pi**(3.))*(G**5.)*(M**5.))
    const = (constant*constant2).to(u.s**(10)/u.m**(10))
    result = ((7*math.pi*(phi(r)**6.))/(2.**(17./2.)))
    return ((result*const).to(u.km**(2.)/(u.s**(2.))))**(1./2.)
print(n_i(1*u.parsec))

42202.32731986435 1 / pc3


In [81]:
def GAMMA(r1):
    a=(r1**(2.)/((R.value)**(2.)))
    b=(r0**(2.)/((R.value)**(2.)))
    return r1**(2.)*((1.+(a))**(-9./4.))*(1./((1. + b)**(1./4.)) + 1./((1. + a)**(1./4.)))**(3./7.) - abs((1./((1. + b)**(1./4.)) - 1./((1. + a)**(1./4.))))**(3./7.)

In [82]:
#CONSTANTS
#sigma = 2*math.pi*(85.*math.pi/(6.*math.sqrt(2)))**(2./7.) * (G**(2.) * (m_i+m_j)**(10./7.) * m_i**(2./7.) * m_j**(2./7.))/(c**(10./7.))
alfa = ((4*24*(2**(1/2))*N_i*(R**(2.)))/(math.pi*(G**(5.))*(M**(5.))*(2**(17./2.))))
betha = ((3.*M)/(m_i*4*math.pi*(R**(3.))))
etha = (G*M/R)
zetha = alfa**(-1./2.) * betha**(3./2.) * etha**(-3.) * alfa**(3./14.) * etha**(9./7.) * betha**(-3./14.)
#print(zetha)

In [83]:
r = np.linspace(.1, 1., 10)
for i in r:
    r0 = i
    I, err = quad(GAMMA, i-0.1, i+0.1)
    print("%.2f"%r0,"\t",I)

0.10 	 -0.010762505505623694
0.20 	 -0.008672081755758903
0.30 	 -0.0021532899709860117
0.40 	 0.006036504926768599
0.50 	 0.01387967497409903
0.60 	 0.02012870918596568
0.70 	 0.024270834711567477
0.80 	 0.02633219142923157
0.90 	 0.02664021087714669
1.00 	 0.02563121938631204


In [84]:
def GAMMA(r1,r0): #Equation (11)
    a=(r1**(2.)/((R.value)**(2.)))
    b=(r0**(2.)/(R**(2.)))
    return (r1**(2.))*((1.+(a))**(-9./4.))*(1./((1. + a)**(1./4.)) + 1./((1. + b)**(1./4.)))**(3./7.) - abs((1./((1. + a)**(1./4.)) - 1./((1. + b)**(1./4.))))**(3./7.)

r = np.linspace(.01, 1., 10)
for i in r:
    r0 = i*u.parsec
    I, err = quad(GAMMA, 0.1-i, i+0.1, args=(r0,))
    nash_rubi = (14*math.pi/3.)*sigma*(n_j(r0)/vel_j(r0))*zetha
    #I = I*units
    merger = (I*nash_rubi).to((u.parsec**(-3.)*(u.yr**(-1))))
    print("%.2f"%r0.value,"\t",merger)

0.01 	 -7.40866213587862e-11 1 / (pc3 yr)
0.12 	 -8.22129240574852e-10 1 / (pc3 yr)
0.23 	 -2.2681366524064596e-09 1 / (pc3 yr)
0.34 	 -3.731875935894959e-09 1 / (pc3 yr)
0.45 	 -4.782038152749052e-09 1 / (pc3 yr)
0.56 	 -5.3195141039395515e-09 1 / (pc3 yr)
0.67 	 -5.435188430287096e-09 1 / (pc3 yr)
0.78 	 -5.272715184860336e-09 1 / (pc3 yr)
0.89 	 -4.9553773014764494e-09 1 / (pc3 yr)
1.00 	 -4.56701940144206e-09 1 / (pc3 yr)


In [85]:
kappa = (1-((8*24*(2**(1./2.))*N_i*m_i)/(3*M*2**(17./2.))))*(1*u.M_sun)
r_star_quad = (4.*(kappa**(2.))*(M*R)**(2.))/((m_i*N_i)**(4.)) - R**(2.)
r_star = (abs(r_star_quad))**(1./2.)
print(r_star)

0.9798978263063961 pc


In [86]:
E_value = phi(r_star) - ((1./2.)*vel_j(r_star)**(2.))
print(E_value.to(u.parsec**2./u.s**2.))

3.2182719349274315e-25 pc2 / s2


In [87]:
def GAMMA(r1,r0): #Equation (11)
    a=(r1**(2.)/((R.value)**(2.)))
    b=(r0**(2.)/(R**(2.)))
    return (r1**(2.))*((1.+(a))**(-9./4.))*(1./((1. + a)**(1./4.)) + 1./((1. + b)**(1./4.)))**(3./7.) - abs((1./((1. + a)**(1./4.)) - 1./((1. + b)**(1./4.))))**(3./7.)



print("radius \t merger_rate \t\t\t Number_Mergers at Universe's time \t time to the first Merger")
r = np.linspace(.1, 1., 20)
colm1 = []
colm2 = []
colm3 = []
colm4 = []
for i in r:
    r0 = i*u.parsec
    colm1.append("%.2f"%r0.value)
    I, err = quad(GAMMA, 1., 0.01, args=(r0,))
    nash_rubi = (14*math.pi/3.)*sigma*(n_j(r0)/vel_j(r0))*zetha*u.parsec**(3)
    #I = I*units
    merger = (I*nash_rubi).to((u.yr**(-1)))
    colm2.append(merger.value)
    colm3.append((merger*(cosmo.age(0).to(u.yr))).value)
    colm4.append(((merger**(-1.)).to(u.Gyr)).value)
    print("%.2f"%r0.value,"\t",merger,"\t\t", merger*(cosmo.age(0).to(u.yr)),"\t\t\t",(merger**(-1.)).to(u.Gyr))#/(cosmo.age(0).to(u.yr)))

radius 	 merger_rate 			 Number_Mergers at Universe's time 	 time to the first Merger
0.10 	 5.267248732429633e-09 1 / yr 		 72.61895413116338 			 0.18985243545518465 Gyr
0.15 	 4.905542609592048e-09 1 / yr 		 67.63215330256695 			 0.2038510475975993 Gyr
0.19 	 4.51838483084413e-09 1 / yr 		 62.294453413188556 			 0.2213180234613126 Gyr
0.24 	 4.133238723658609e-09 1 / yr 		 56.98448820890625 			 0.24194102176484797 Gyr
0.29 	 3.769912685003538e-09 1 / yr 		 51.97535378673524 			 0.26525813289467776 Gyr
0.34 	 3.441591329873097e-09 1 / yr 		 47.448824921351445 			 0.2905632610472881 Gyr
0.38 	 3.1557329276767907e-09 1 / yr 		 43.50772791765521 			 0.31688359659009135 Gyr
0.43 	 2.9150801914777876e-09 1 / yr 		 40.189876246064756 			 0.3430437361289379 Gyr
0.48 	 2.718764719279115e-09 1 / yr 		 37.48329734785178 			 0.36781410061299885 Gyr
0.53 	 2.563399399136006e-09 1 / yr 		 35.34129349912872 			 0.39010698072920286 Gyr
0.57 	 2.4440756102651893e-09 1 / yr 		 33.696190108165545 			 0

In [88]:
print(colm1)

['0.10', '0.15', '0.19', '0.24', '0.29', '0.34', '0.38', '0.43', '0.48', '0.53', '0.57', '0.62', '0.67', '0.72', '0.76', '0.81', '0.86', '0.91', '0.95', '1.00']


In [89]:
f = open("MR_T0.1.txt", "w")
f.write("Mpbh = 0.1 u.Solar Mass")
f.write("\n")
f.write("radius \t merger_rate \t Number_Mergers at Universe's time \t time to the first Merger")
f.write("\n")
for i in range(len(r)):
    print(colm1[i],"\t ",colm2[i],"\t ",colm3[i],"\t ",colm4[i], file=f)
f.close()

In [90]:
read_file = pd.read_csv (r"MR_T0.1.txt")
read_file.to_csv (r"MR_T0.1.csv", index=None)